In [8]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [9]:
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [10]:
df = df.dropna()
df = shuffle(df, random_state=2)
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [11]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 1s 5ms/step - loss: 1.4593 - mae: 0.9382 - mse: 1.4593 - val_loss: 1.3196 - val_mae: 0.8948 - val_mse: 1.3196
Epoch 2/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3537 - mae: 0.9097 - mse: 1.3537 - val_loss: 1.6363 - val_mae: 1.0320 - val_mse: 1.6363
Epoch 3/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3262 - mae: 0.9011 - mse: 1.3262 - val_loss: 1.8492 - val_mae: 1.0666 - val_mse: 1.8492
Epoch 4/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3046 - mae: 0.8946 - mse: 1.3046 - val_loss: 1.6488 - val_mae: 1.0095 - val_mse: 1.6488
Epoch 5/10
209/209 [==============================] - 1s 3ms/step - loss: 1.2976 - mae: 0.8897 - mse: 1.2976 - val_loss: 1.2303 - val_mae: 0.8755 - val_mse: 1.2303
Epoch 6/10
209/209 [==============================] - 1s 3ms/step - loss: 1.2640 - mae: 0.8834 - mse: 1.2640 - val_loss: 1.2360 - val_mae: 0.8690 - val_mse: 1.2360
Epoch 7/10
209/2

In [ ]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()